In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:4 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:5 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main Sources [2,164 kB]
Ign:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Relea

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-10-16 13:56:16--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.58MB/s    in 0.6s    

2022-10-16 13:56:18 (1.58 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Gift_Card_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   24371595|R27ZP1F1CD0C3Y|B004LLIL5A|     346014806|Amazon eGift Card...|       Gift Card|          5|            0|          0|   N|                Y|          Five Stars|Great birthday gi...|2015-08-31 00:00:00|
|         US|   42489718| RJ7RSBCHUDNNE|B004LLIKVU|     473048287|Amazon.com eGift ...| 

### Create DataFrames to match tables

In [5]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
review_id_df = df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_df.show()

+--------------+-----------+----------+--------------+-------------------+
|     review_id|customer_id|product_id|product_parent|        review_date|
+--------------+-----------+----------+--------------+-------------------+
|R27ZP1F1CD0C3Y|   24371595|B004LLIL5A|     346014806|2015-08-31 00:00:00|
| RJ7RSBCHUDNNE|   42489718|B004LLIKVU|     473048287|2015-08-31 00:00:00|
|R1HVYBSKLQJI5S|     861463|B00IX1I3G6|     926539283|2015-08-31 00:00:00|
|R2HAXF0IIYQBIR|   25283295|B00IX1I3G6|     926539283|2015-08-31 00:00:00|
| RNYLPX611NB7Q|     397970|B005ESMGV4|     379368939|2015-08-31 00:00:00|
|R3ALA9XXMBEDZR|   18513645|B004KNWWU4|     326384774|2015-08-31 00:00:00|
|R3R8PHAVJFTPDF|   22484620|B004LLIKVU|     473048287|2015-08-31 00:00:00|
|R18WWEK8OIXE30|   14765851|BT00CTP2EE|     775486538|2015-08-31 00:00:00|
|R1EGUNQON2J277|   18751931|B004LLIKVU|     473048287|2015-08-31 00:00:00|
|R21Z4M4L98CPU2|   15100528|B004W8D102|     595099956|2015-08-31 00:00:00|
| R6JH7A117FHFA|    35597

In [6]:
# Create the customers_table DataFrame
customers_df = df.groupby("customer_id").agg({"customer_id": "count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   12819130|             2|
|   50079826|             2|
|    3399669|             1|
|   23344898|             1|
|   42139031|             1|
|   48994143|             1|
|     768171|             1|
|   29907188|             1|
|   28161223|             1|
|     184483|             1|
|     225566|             1|
|   42628722|             1|
|    2903234|             1|
|   11285702|             2|
|   15423461|             1|
|   30717827|             1|
|   35594973|             1|
|   20157057|             1|
|   27747830|             1|
|   11150152|             1|
+-----------+--------------+
only showing top 20 rows



In [7]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(["product_id", "product_title"]).drop_duplicates()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00CHQKNUM|Amazon.com Gift C...|
|B00CRQ4ZOM|Amazon Gift Card ...|
|B00H5BMZ32|Amazon eGift Card...|
|BT00CTP84S|Amazon.com Gift C...|
|B004LLILN2|Amazon eGift Card...|
|B00N59437O|Amazon Gift Card ...|
|B005J0444E|Amazon.com Gift C...|
|B00E1QB2K6|Amazon Gift Card ...|
|B004LLJ6IQ|Amazon Gift Card ...|
|B004LLILQ4|Amazon eGift Card...|
|B0091JL2CQ|Amazon.com Gift C...|
|B0062ONNL6|Amazon Gift Card ...|
|BT00CTP0N2|Amazon.com Gift C...|
|B009I1Z11U|Amazon Video Gift...|
|B009LLWBTM|Amazon Video Gift...|
|B00BWDHS04|Amazon Gift Card ...|
|B0091JKZYW|Amazon.com Gift C...|
|B004LLJ6C2|Amazon Gift Card ...|
|B007V6ETDK|Amazon eGift Card...|
|B005Z3D5OU|Amazon.com Gift C...|
+----------+--------------------+
only showing top 20 rows



In [8]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = df.select(["review_id", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()

+--------------+-----------+
|     review_id|review_date|
+--------------+-----------+
|R27ZP1F1CD0C3Y| 2015-08-31|
| RJ7RSBCHUDNNE| 2015-08-31|
|R1HVYBSKLQJI5S| 2015-08-31|
|R2HAXF0IIYQBIR| 2015-08-31|
| RNYLPX611NB7Q| 2015-08-31|
|R3ALA9XXMBEDZR| 2015-08-31|
|R3R8PHAVJFTPDF| 2015-08-31|
|R18WWEK8OIXE30| 2015-08-31|
|R1EGUNQON2J277| 2015-08-31|
|R21Z4M4L98CPU2| 2015-08-31|
| R6JH7A117FHFA| 2015-08-31|
|R1XZHS8M1GCGI7| 2015-08-31|
|R1DAI0N03SKRJN| 2015-08-31|
|R2F6SKZOEYQRU3| 2015-08-31|
| RIBOP6OEAZA47| 2015-08-31|
|R15H8E7WD6XD29| 2015-08-31|
| RVN4P3RU4F8IE| 2015-08-31|
| RCS8F9JCAAXC7| 2015-08-31|
| R6811C4E7UYL2| 2015-08-31|
|R196T0NZ73TYIM| 2015-08-31|
+--------------+-----------+
only showing top 20 rows



In [9]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R27ZP1F1CD0C3Y|          5|            0|          0|   N|
| RJ7RSBCHUDNNE|          5|            0|          0|   N|
|R1HVYBSKLQJI5S|          5|            0|          0|   N|
|R2HAXF0IIYQBIR|          1|            0|          0|   N|
| RNYLPX611NB7Q|          5|            0|          0|   N|
|R3ALA9XXMBEDZR|          5|            0|          0|   N|
|R3R8PHAVJFTPDF|          5|            0|          0|   N|
|R18WWEK8OIXE30|          5|            0|          0|   N|
|R1EGUNQON2J277|          1|            0|          0|   N|
|R21Z4M4L98CPU2|          5|            0|          0|   N|
| R6JH7A117FHFA|          5|            0|          0|   N|
|R1XZHS8M1GCGI7|          5|            1|          1|   N|
|R1DAI0N03SKRJN|          5|            1|          1|   N|
|R2F6SKZOEYQRU3|          5|            

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [36]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://dataviz.cds7xvozbpk0.us-east-2.rds.amazonaws.com:5432/amazon_reviews"
config = {"user":"postgres", 
          "password": <"password">, 
          "driver":"org.postgresql.Driver"}

In [37]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

Py4JJavaError: ignored

In [18]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

Py4JJavaError: ignored

In [19]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

Py4JJavaError: ignored

In [20]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

Py4JJavaError: ignored